In [1]:
#| label: python_clases_codigo
#| eval: false

import math
import folium # Librería indispensable en Python para generar mapas interactivos

# --- 0. TRAEMOS NUESTRA MÁQUINA MATEMÁTICA ---
# Pegamos aquí la función que ya habíamos creado para que este código funcione por sí solo
def haversine(lat1, lon1, lat2, lon2, radio=6371.0):
    dlat, dlon = math.radians(lat2 - lat1), math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    return radio * (2 * math.atan2(math.sqrt(a), math.sqrt(1 - a)))


# --- 1. DEFINICIÓN DEL MOLDE (LA CLASE) ---
# Usamos la palabra reservada 'class' seguida del nombre (en mayúscula por convención)
class PuntoEspacial:
    
    # [1 📍] 1.1 EL CONSTRUCTOR (__init__)
    # Esta función especial se ejecuta automáticamente el momento en que "nace" un punto.
    # 'self' significa "yo mismo". Es la forma en que el objeto se refiere a sus propios datos.
    # Definimos que latitud y longitud son obligatorios, pero el nombre tiene un valor por defecto.
    def __init__(self, latitud, longitud, nombre="Punto sin nombre"):
        
        # Guardamos la latitud que nos entregaron dentro del cuerpo del objeto (self)
        self.latitud = latitud
        
        # Guardamos la longitud dentro del objeto
        self.longitud = longitud
        
        # Guardamos el nombre dentro del objeto
        self.nombre = nombre

    # [2 💬] 1.2 LA REPRESENTACIÓN TEXTUAL (__str__)
    # Si le decimos a Python print(objeto), por defecto imprimirá basura técnica.
    # Esta función especial le enseña a Python cómo queremos que se lea el objeto en pantalla.
    def __str__(self):
        # SACA: Un texto bonito combinando el nombre y las coordenadas
        return f"[{self.nombre}] (Lat: {self.latitud}, Lon: {self.longitud})"

    # [3 📐] 1.3 LOS MÉTODOS (COMPORTAMIENTO ANALÍTICO)
    # Es una función normal, pero como tiene 'self', vive dentro del objeto.
    # Agregamos el parámetro 'radio' para que el usuario pueda pedir el resultado en metros.
    def distancia_hacia(self, otro_punto, radio=6371.0):
        
        # Llamamos a nuestra función haversine externa pasándole el radio solicitado
        # Le enviamos NUESTRAS coordenadas (self) y las coordenadas DEL OTRO (otro_punto)
        distancia = haversine(self.latitud, self.longitud, otro_punto.latitud, otro_punto.longitud, radio)
        
        # SACA: La distancia numérica calculada entre los dos objetos
        return distancia
        
    # [4 🗺️📐] 1.4 EL MÉTODO GRÁFICO (MAPA BASE CON ETIQUETA VISIBLE)
    # Este método le da la habilidad al objeto de dibujarse en un mapa web interactivo.
    def graficar_ruta(self, otro_punto):
        
        # Primero, le decimos al objeto que calcule su propia distancia hacia el destino
        distancia_calculada = self.distancia_hacia(otro_punto)
        
        # Calculamos el punto medio geométrico para centrar la cámara y poner el texto
        centro_lat = (self.latitud + otro_punto.latitud) / 2
        centro_lon = (self.longitud + otro_punto.longitud) / 2
        
        # Creamos el lienzo del mapa base (estilo OpenStreetMap por defecto)
        mapa = folium.Map(location=[centro_lat, centro_lon], zoom_start=6)
        
        # Ponemos un pin en nuestra ubicación de origen (self)
        folium.Marker([self.latitud, self.longitud], popup=self.nombre).add_to(mapa)
        
        # Ponemos un pin en el destino (otro_punto)
        folium.Marker([otro_punto.latitud, otro_punto.longitud], popup=otro_punto.nombre).add_to(mapa)
        
        # Trazamos una línea roja que conecte ambas coordenadas
        folium.PolyLine([(self.latitud, self.longitud), (otro_punto.latitud, otro_punto.longitud)], color="red", weight=3).add_to(mapa)
        
        # ¡NUEVO!: Ponemos un marcador transparente en el medio con un 'Tooltip' permanente.
        # Esto obliga a Folium a mostrar el texto de la distancia sin necesidad de hacer clic.
        etiqueta = f"Distancia: {distancia_calculada:.2f} km"
        folium.Marker(
            [centro_lat, centro_lon],
            icon=folium.DivIcon(html=""), # Ocultamos el ícono tradicional
            tooltip=folium.Tooltip(etiqueta, permanent=True, direction="top")
        ).add_to(mapa)
        
        # SACA: El objeto mapa listo para ser mostrado
        return mapa


# --- 2. CREACIÓN DE OBJETOS (INSTANCIACIÓN) ---

print("--- Creando objetos a partir del molde ---")

# [1 📍] INSTANCIACIÓN: Creamos el primer objeto (Bogotá). 
# Noten que no le pasamos el parámetro 'self', Python hace eso invisiblemente.
punto_origen = PuntoEspacial(4.6097, -74.0817, "Bogotá")

# Creamos el segundo objeto (Medellín). Usamos los mismos planos, pero con otros datos.
punto_destino = PuntoEspacial(6.2442, -75.5812, "Medellín")

# Creamos un tercer objeto, pero omitimos el nombre para ver si funciona el valor por defecto
punto_misterioso = PuntoEspacial(12.5847, -81.7006)

# [2 💬] REPRESENTACIÓN: Probamos nuestra función especial __str__ imprimiendo los objetos
print(punto_origen)
print(punto_destino)
print(punto_misterioso)

print("\n--- Usando los métodos del objeto ---")

# [3 📐] MÉTODOS ANALÍTICOS: Le pedimos al objeto que mida la distancia hacia 'punto_destino'
resultado_km = punto_origen.distancia_hacia(punto_destino)
print(f"La distancia desde {punto_origen.nombre} hasta {punto_destino.nombre} es: {resultado_km:.2f} km")

# Truco de conversión: Pedimos el resultado en metros modificando el parámetro 'radio' a 6371000.0
resultado_metros = punto_origen.distancia_hacia(punto_destino, radio=6371000.0)
print(f"O expresado en metros: {resultado_metros:.2f} m")

# [4 🗺️📐] MÉTODO GRÁFICO: Finalmente, le pedimos al objeto que dibuje el mapa de su ruta
mapa_final = punto_origen.graficar_ruta(punto_destino)
print("\n¡Mapa interactivo generado con éxito en la variable 'mapa_final'!")
# Nota: Si estás en un Notebook, basta con escribir 'mapa_final' en una celda para visualizarlo.

--- Creando objetos a partir del molde ---
[Bogotá] (Lat: 4.6097, Lon: -74.0817)
[Medellín] (Lat: 6.2442, Lon: -75.5812)
[Punto sin nombre] (Lat: 12.5847, Lon: -81.7006)

--- Usando los métodos del objeto ---
La distancia desde Bogotá hasta Medellín es: 246.14 km
O expresado en metros: 246135.99 m

¡Mapa interactivo generado con éxito en la variable 'mapa_final'!


In [2]:
mapa_final